# Learning Objectives

In this lab we are going to:

*   Explore POS Tagging using NLTK
*   Learn about Hidden Markov Models (HMM)
*   Perform POS tagging with HMM

In [ ]:
# Installing necessary packages from NLTK
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# POS Tagging 
POS tagging is the process of assigning a part-of-speech label to
each word in an input text, where the tagging model takes a sequence of words and a tagset as input and gives the output as a sequence of tags one per token. There are various parts of speech tagsets. The most common tagsets are:

1- <a href= "http://ucrel.lancs.ac.uk/claws5tags.html">Claws5</a>: 62 different tags <br>
2- <a href="https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html">Penn Treebank</a>: 45 different tags <br>
3- <a href = "https://en.wikipedia.org/wiki/Brown_Corpus">The Brown Corpus tagset</a>: (87 tags)<br>
4- <a href = "https://universaldependencies.org/u/pos/">UD tagset</a>

## Approaches

POS tagging can be done using different approaches such as:
 

*   Pointwise prediction: a classifier that predicts each word individually such as perceptron.

*   Generative sequence models: a probabilistic model that assigns probabilities to sequences of words such as Hidden Markov Model.


*   Discriminative sequence models: predict whole sequence with a classifier such as conditional random fields (CRF).


# NLTK POS Tagging

The NLTK tagger can be used as follows:


In [ ]:
from nltk.tokenize import word_tokenize
# tokenize the sentence before POS tagging
text = word_tokenize("And is it very interesting for everyone?")
nltk.pos_tag(text)

[('And', 'CC'),
 ('is', 'VBZ'),
 ('it', 'PRP'),
 ('very', 'RB'),
 ('interesting', 'VBG'),
 ('for', 'IN'),
 ('everyone', 'NN'),
 ('?', '.')]

The brown corpus has been manually tagged with part-of-speech tags which is useful for testing taggers and for training statistical taggers. In order to read a tagged corpus we can use:

In [ ]:
from nltk.corpus import brown
# Accessing manually tagged brown corpus
print (brown.tagged_words())

[('The', 'AT'), ('Fulton', 'NP-TL'), ...]


## Exercise 1:
Get the count of each POS tag assigned to the word **(ignore case)** "_dog_" in the **news** category of the Brown corpus.

In [ ]:
# Slicing brown corpus with news category
tagged_words = brown.tagged_words(categories='news')
tag_count_map = {}

# get frequency
for word,tag in tagged_words:
    # checking if "dog" is tagged in the brown corpus
    if word.lower() == "dog":
        if tag in tag_count_map.keys():
            tag_count_map[tag] = tag_count_map[tag] + 1 # increasing the count by 1 if the tag is already present
        else:
            tag_count_map[tag] = 1 # adding new entry in the dictionary
# print
tag_count_map

{'NN': 7}

In [ ]:
# one-liner implementation using Counter and list comprehension
from collections import Counter
Counter([tag for (word, tag) in brown.tagged_words(categories=['news']) if word.lower() == 'dog'])

Counter({'NN': 7})

In [ ]:
# using ConditionalFreqDist in nltk
tagged_words = brown.tagged_words(categories=['news'])
# lower case 'dog'
tagged_words = [(word.lower(), tag) for (word, tag) in tagged_words]
# use cond. freq. dist. given the tag
cfd = nltk.ConditionalFreqDist(tagged_words)
cfd['dog']

FreqDist({'NN': 7})

## Exercise 2:
Find the frequency distribution of each tag in the brown corpus.

In [ ]:
tagged_words = brown.tagged_words()
counts = dict()

for word, tag in tagged_words:
  if tag in counts:
    counts[tag] += 1
  else:
    counts[tag] = 1

sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
sorted_counts[:5]

[('NN', 152470), ('IN', 120557), ('AT', 97959), ('JJ', 64028), ('.', 60638)]

In [ ]:
# using Counter and map
Counter(map(lambda x: x[1], brown.tagged_words())).most_common(5)

[('NN', 152470), ('IN', 120557), ('AT', 97959), ('JJ', 64028), ('.', 60638)]

In [ ]:
# using FreqDist in nltk
tagged_words = brown.tagged_words()
words, tags = zip(*tagged_words)
freq_dist = nltk.FreqDist(tags)
freq_dist.most_common(5)

[('NN', 152470), ('IN', 120557), ('AT', 97959), ('JJ', 64028), ('.', 60638)]

## Exercise 3:

What are the most common verbs in **fiction** category in the brown corpus? 

In [ ]:
# your code goes here;

verb_tags = ['VB', 'VBN', 'VBD', 'VBG', 'VBZ']
tagged_words = brown.tagged_words(categories=['fiction'])
counts = dict()

for word, tag in tagged_words:
  if tag in verb_tags:
    if (word, tag) in counts:
      counts[(word, tag)] += 1
    else:
      counts[(word, tag)] = 1

sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
sorted_counts[:5]

[(('said', 'VBD'), 177),
 (('came', 'VBD'), 91),
 (('went', 'VBD'), 79),
 (('get', 'VB'), 78),
 (('know', 'VB'), 74)]

In [ ]:
# one-liner using Counter and list comprehension
Counter([(word, tag) for (word, tag) in brown.tagged_words(categories=['fiction']) if tag.startswith('VB')]).most_common(5)

[(('said', 'VBD'), 177),
 (('came', 'VBD'), 91),
 (('went', 'VBD'), 79),
 (('get', 'VB'), 78),
 (('know', 'VB'), 74)]

In [ ]:
# using FreqDist in nltk
fiction_tagged_words = brown.tagged_words(categories='fiction')
fiction_tagged_words = [(word, tag) for (word, tag) in fiction_tagged_words if tag.startswith('VB')]
freq_dist = nltk.FreqDist(fiction_tagged_words)
freq_dist.most_common(5)

[(('said', 'VBD'), 177),
 (('came', 'VBD'), 91),
 (('went', 'VBD'), 79),
 (('get', 'VB'), 78),
 (('know', 'VB'), 74)]

# Hidden Markov Model

The sequence of tags can be viewed as a Markov chain so let us explore the construction and solution of a Hidden Markov Model. 

An HMM has two components:

*   **Transition Probabilities** which represents the probability of a tag occurring given the previous tag i.e. $P(t_i|t_{i-1})$.
  * For Example, modal verbs (`MD`) like *will* are very likely to be followed by a verb in the base form, a `VB`, like *race*, therefore it is more likely that modal verbs will occur with main verb.
  * We compute the maximum likelihood estimate of this transition probability by counting, out of the times we see the first tag in a labeled corpus, how often the first tag is followed by the second:
  $$
  \begin{equation}
  P(t_{i} | t_{i-1}) = \frac{C(t_{i-1}, t_{i})}{C(t_{i-1})} \\
  P(MD | VB) = \frac{C(MD, VB)}{C(MD)}
  \end{equation}
  $$

*   **Emission Probabilities** represents the probability, given a tag that it will be associated with a given word i.e. $P(w_i|t_i)$.
  * For Example, probability of a given tag `MD` associated with the word *will* is:
  $$
  \begin{equation}
  P(w_i|t_i) = \frac{C(t_i, w_i)}{C(t_i)} \\
  P(will|MD) = \frac{C(MD, \text{will})}{C(MD)}
  \end{equation}
  $$

---

## Exercise 4: 

Consider that we have an HMM with hidden states Noun, Verb, Adj and the following transition probability where $p(Y_{i+1}|Y_i)$ is the probability of state $Y_{i+1}$ occuring after $Y_i$ and the table of probabilities is as follows:

| $p(Y_{i+1}|Y_i)$ | $Y_{i+1}$=Start | $Y_{i+1}$=Noun | $Y_{i+1}$=Verb | $Y_{i+1}$=Adj |
|:-----------------|:-----------------|:--------------:|:--------------:|:-------------:|
| $Y_i$=Start      | 0.0      |  0.5           |  0.4           | 0.1           |
| $Y_i$=Noun       | 0.0       |  0.3           |  0.5           | 0.2           |
| $Y_i$=Verb       | 0.0       |  0.7           |  0.2           | 0.1           |
| $Y_i$=Adj        | 0.0        |  0.8           |  0.1           | 0.1           |

Furthermore, consider that the model has a vocabulary as follows, with the probability of $p(X_i|Y_i)$ as follows 

| $p(X_i|Y_i)$ | cats | dogs | drink | water | milk | fresh |
|:-------------|:----:|:----:|:-----:|:-----:|:----:|:-----:|
| $Y_i$=Noun   | 0.2  | 0.2  |  0.2  | 0.2   | 0.1  | 0.0   |
| $Y_i$=Verb   | 0.1  | 0.1  | 0.4   | 0.2   | 0.1  | 0.1   |
| $Y_i$=Adj    | 0.0  | 0.0  | 0.2   | 0.0   | 0.2  | 0.8   |


Implement the above table and write a function that takes a sequence of words and a sequence of part-of-speech tags and returns the probability using the above model. Calculate the probability of the sentence "*cats drink fresh milk*" given the tags "*noun verb adj verb*"

In [ ]:
all_tags = ["start","noun","verb","adj"]
all_words = ["cats","dogs","drink","water","milk","fresh"]

In [ ]:
transitions = {
  'start': {'noun': 0.5, 'verb': 0.4, 'adj': 0.1, 'start': 0.0},
  'noun': {'noun': 0.3, 'verb': 0.5, 'adj': 0.2, 'start': 0.0},
  'verb': {'noun': 0.7, 'verb': 0.2, 'adj': 0.1, 'start': 0.0},
  'adj': {'noun': 0.8, 'verb': 0.1, 'adj': 0.1, 'start': 0.0},
}

emissions = {
  'noun': {'cats': 0.2, 'dogs': 0.2, 'drink': 0.2, 'fresh': 0.0, 'milk': 0.1, 'water': 0.2},
  'verb': {'cats': 0.1, 'dogs': 0.1, 'drink': 0.4, 'fresh': 0.1, 'milk': 0.1, 'water': 0.2},
  'adj': {'cats': 0.0, 'dogs': 0.0, 'drink': 0.2, 'fresh': 0.8, 'milk': 0.2, 'water': 0.0},
  'start': {'cats': 0.0, 'dogs': 0.0, 'drink': 0.0, 'fresh': 0.0, 'milk': 0.0, 'water': 0.0},
}

In [ ]:
def hmm(words, tags):
    prob = 1.0
    
    prev_tag = 'start'   # 'start' is same as t0 in lecture slides

    for tag, word in zip(tags, words):
      prob = prob * transitions[prev_tag][tag] * emissions[tag][word]
      prev_tag = tag

    return prob

print(hmm(["cats","drink","fresh","milk"], ["noun","verb","adj","verb"]))

1.6000000000000006e-05


## Exercise 5 
Write a function that learns the emission and transition probabilities for the Hidden Markov Model using the tagged corpus given below.

In [ ]:
sentences = [
    ["cats","drink","milk"],
    ["dogs","drink","water"],
    ["fresh","milk"],
    ["dogs","drink","fresh","milk"],
    ["cats","milk"]
]

tagged = [
    ["noun","verb","noun"],
    ["noun","verb","noun"],
    ["adj","noun"],
    ["noun","verb","adj","noun"],
    ["noun","noun"]
]

In [ ]:
def hmm_learn(sentences, tagged):
    # Reserve a dictionary for transition and emission probabilities
    transitions = {t:{t2:0.0 for t2 in all_tags} for t in all_tags}
    emissions    = {t:{w:0.0 for w in all_words} for t in all_tags}
    # Reserve a dictionary for transition and emission counts
    transitions_counts = {t:{t2:0.0 for t2 in all_tags} for t in all_tags}
    emissions_counts = {t:{w:0.0 for w in all_words} for t in all_tags}

    # Iterate through sentence and tag pair
    for sents, tags in zip(sentences, tagged):
      prev = "start"
      # Iterated through word and tag pair
      for word, tag in zip(sents, tags):
        # Update the transitions and emission counts
        transitions_counts[prev][tag] += 1
        emissions_counts[prev][word] += 1
        prev = tag

    # Calculate the transitions_counts total and emissions_counts total by adding counts
    for tag in all_tags:
      transitions_total = sum(transitions_counts[tag].values())
      emissions_total = sum(emissions_counts[tag].values())

      if transitions_total > 0:  # avoid dividing by zero
        # normalize the tag counts
        for next_tag in all_tags:
          transitions[tag][next_tag] = transitions_counts[tag][next_tag] / transitions_total

      if emissions_total > 0:  # avoid dividing by zero
        # normalize the word counts
        for word in all_words:
          emissions[tag][word] = emissions_counts[tag][word] / emissions_total

    return transitions, emissions

transitions, emissions = hmm_learn(sentences, tagged)

import pprint
print('Transitions')
pprint.pprint(transitions)
print('\n')
print('Emissions')
pprint.pprint(emissions)

Transitions
{'adj': {'adj': 0.0, 'noun': 1.0, 'start': 0.0, 'verb': 0.0},
 'noun': {'adj': 0.0, 'noun': 0.25, 'start': 0.0, 'verb': 0.75},
 'start': {'adj': 0.2, 'noun': 0.8, 'start': 0.0, 'verb': 0.0},
 'verb': {'adj': 0.3333333333333333,
          'noun': 0.6666666666666666,
          'start': 0.0,
          'verb': 0.0}}


Emissions
{'adj': {'cats': 0.0,
         'dogs': 0.0,
         'drink': 0.0,
         'fresh': 1.0,
         'milk': 0.0,
         'water': 0.0},
 'noun': {'cats': 0.2222222222222222,
          'dogs': 0.2222222222222222,
          'drink': 0.0,
          'fresh': 0.0,
          'milk': 0.4444444444444444,
          'water': 0.1111111111111111},
 'start': {'cats': 0.0,
           'dogs': 0.0,
           'drink': 0.0,
           'fresh': 0.0,
           'milk': 0.0,
           'water': 0.0},
 'verb': {'cats': 0.0,
          'dogs': 0.0,
          'drink': 1.0,
          'fresh': 0.0,
          'milk': 0.0,
          'water': 0.0}}
